In [1]:
import os; os.environ['RUST_BACKTRACE'] = '1'

In [2]:
import healpy
import numpy as np
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery
from joblib import Parallel, delayed

from mom_builder import mom_from_array, mom_from_batch_it
from multiorder import AbstractMultiorderMapBuilder
from min_max_mean_state import MinMaxMeanState, MinMaxMeanStateMerger
from paths import *

In [3]:
class StateMerger(MinMaxMeanStateMerger):
    def __init__(self, threshold: float = 0.16):
        self.threshold = threshold
    
    def validate_state(self, state: MinMaxMeanState) -> bool:
        norm = max(map(abs, [state.min, state.max]))
        if norm == 0.0:
            return True
        return (state.max - state.min) / norm <= self.threshold
    
sfd_query = SFDQuery(INPUT_DIR)

def ebv(norder: int, index_range=None):
    n_size = healpy.order2nside(norder)
    n_pix = healpy.order2npix(norder)
    
    if index_range is None:
        index_range = (0, n_pix)
    if index_range[1] > n_pix:
        index_range = (index_range[0], n_pix)
    
    indexes = np.arange(*index_range)
    
    ra, dec = healpy.pix2ang(n_size, indexes, nest=True, lonlat=True)
    coord = SkyCoord(ra=ra, dec=dec, unit='deg')
    
    return sfd_query(coord)


def ebv_f64(norder: int, index_range=None):
    return ebv(norder, index_range).astype(np.float64)
    

class Builder(StateMerger, AbstractMultiorderMapBuilder):
    def __init__(self, max_norder, threshold, ebv_vals):
        AbstractMultiorderMapBuilder.__init__(self, max_norder)
        StateMerger.__init__(self, threshold)
        
        self.ebv = ebv_vals
        
    def calculate_state(self, index_max_norder: int) -> MinMaxMeanState:
        value = self.ebv[index_max_norder].item()
        return MinMaxMeanState(min=value, max=value, mean=value)

In [4]:
max_norder = 9

%time ebv_vals = ebv(max_norder)

%time tiles = Builder(max_norder=max_norder, threshold=0.16, ebv_vals=ebv_vals).build()

%time mom = mom_from_array(ebv_vals.astype(np.float64), max_norder, 0.16)

CPU times: user 618 ms, sys: 184 ms, total: 802 ms
Wall time: 1.01 s
CPU times: user 3.74 s, sys: 36 ms, total: 3.77 s
Wall time: 3.8 s
CPU times: user 38.7 ms, sys: 9.77 ms, total: 48.5 ms
Wall time: 50.1 ms


In [14]:
len(tiles[6].indexes), len(mom[6][0])

(318, 318)

In [5]:
%%time

max_norder = 14
thereshold = 0.16 / 2
batch_size = 1 << 20

batches = range(0, healpy.order2npix(max_norder) + batch_size, batch_size)

it = Parallel(n_jobs=-1, return_as="generator", backend="threading")(
    delayed(ebv_f64)(max_norder, rng)
    for rng in zip(batches, batches[1:])
)

mom = mom_from_batch_it(it, max_norder, thereshold)

CPU times: user 29min 25s, sys: 2min 13s, total: 31min 39s
Wall time: 7min 23s


In [6]:
ntiles = sum(len(indexes) for indexes, values in mom)
max_ntiles = healpy.order2npix(max_norder)
print(f"Number of tiles: {ntiles:_d} / {max_ntiles:_d} ({ntiles / max_ntiles:.2%})")

Number of tiles: 25_869_093 / 3_221_225_472 (0.80%)
